<a href="https://colab.research.google.com/github/naoya1110/ai_robotics_lab_2024_hands_on/blob/main/Week03_Simple_SGD_Example__with_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is supplemental material for Week 03, explaining how model parameters are optimized using the stochastic gradient descent (SGD) method.

In this example, we will implement a simple linear regression model with PyTorch.

First, let's import the PyTorch, NumPy, and Matplotlib packages.

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

Here, we create a dataset consisting of $x$ (inputs) and $y$ (outputs) using a simple linear equation below. It's important to note that the output data y contains some random noise.

$y = 5x + 3 + \mathrm{noise}$

In [ ]:
x = 10*np.random.rand(100)-5
noise = 3*np.random.randn(x.shape[0])
y = 5*x + 3 + noise

plt.plot(x, y, marker="o", lw= 0, label="data")
plt.xlabel("x")
plt.ylabel("y")
plt.legend()
plt.grid()

Our objective is to discover a linear function model (equation) that can effectively capture the relationship between the $x$ and $y$ dataset.

In fact, instead of employing PyTorch, we can use `np.polyfit()` to fit the dataset. This allows us to acquire the fitting parameters $w$ and $b$ for the linear function $y = wx + b$, where $w$ and $b$ are referred to as the weight and bias, respectively."

In [ ]:
w, b = np.polyfit(x, y, 1)
print(f"w={w:.3f}, b={b:.3f}")

Due to the presence of noise in the $x-y$ dataset, the obtained values of $w$ and $b$ may not match the exact values used to create the dataset. Nevertheless, they are typically close enough to allow us to create a fitting line using these parameters.

In [ ]:
y_fit = w*x + b

plt.plot(x, y, marker="o", lw=0, label="dataset")
plt.plot(x, y_fit, lw=2, label="fit")
plt.xlabel("x")
plt.ylabel("y")
plt.legend()
plt.grid()

While `np.polyfit()` works effectively, in this example, we will achieve the same outcome using PyTorch.

Let's start by converting the $x$ and $y$ dataset into `torch.tensor` objects.

In [ ]:
x = torch.tensor(x)
y = torch.tensor(y)

print(type(x))
print(type(y))

Next, we define a function called `model()` that predicts the output value `y` based on an input value `x` using the parameters `w` and `b`.

In [ ]:
def model(x):
    return w*x + b

We also define a function called `loss_func()` to calculate the mean squared error between `p` and `y`, where `p` represents the output values (predictions) of the `model()`. This type of function is referred to as a loss function, which helps measure the degree of error in the model's predictions.

$\displaystyle \mathrm{loss} = \mathrm{mse}(p, y) = \frac{1}{N}\sum_{i=0}^{N-1}(p_i-y_i)^2$

In [ ]:
def loss_func(p, y):
    return ((p-y)**2).mean()

At this stage, we haven't determined the values of `w` and `b` yet. Therefore, we initialize these values with arbitrary numbers.

In [ ]:
w = torch.tensor(1.0, requires_grad=True)   # you can set any number here
b = torch.tensor(-5.0, requires_grad=True)  # you can set any number here

Now, we can make a prediction using `model()`

In [ ]:
p = model(x)
print(p)

Let's visualize the current prediction. Since we've initialized `w` and `b` with arbitrary numbers, it's expected that the model's prediction won't fit the data well.

In [ ]:
plt.plot(x, y, marker="o", lw=0, label="data")
plt.plot(x, p.detach().numpy(), label="prediction")
plt.grid()
plt.legend()
plt.xlabel("x")
plt.ylabel("y")

Next, we calculate the loss value (mean squared error) using `loss_func()`. It's important to note that the loss value will be very large because `w` and `b` are arbitrary values and have not been optimized yet.

In [ ]:
loss = loss_func(p, y)
print(loss)

To optimize `w` and `b`, we must determine the gradients of the loss with respect to the current values of `w` and `b`. This can be achieved using `loss.backward()`. The gradients, denoted as $\displaystyle \frac{\partial \mathrm{loss}}{\partial w}$ and $\displaystyle \frac{\partial \mathrm{loss}}{\partial b}$, can be accessed via `w.grad` and `b.grad`, respectively.

In [ ]:
loss.backward()
print(w.grad)
print(b.grad)

We can update `w` and `b` using the following equations, where $\eta$ represents the learning rate. This method is known as stochastic gradient descent (SGD):

$\displaystyle w := w - \eta\frac{\partial\mathrm{loss}}{\partial w}$

$\displaystyle b := b - \eta\frac{\partial\mathrm{loss}}{\partial b}$

When updating these values, we don't want to compute gradients. To achieve this, we use with `torch.no_grad()` at the beginning.

In [ ]:
lr = 0.01    # define learning rate

with torch.no_grad():    # disable gradients calculations
    w -= w.grad*lr       # update w
    b -= b.grad*lr       # update b

At this stage, you'll notice that the values of `w` and `b` are closer to their true values ($w$=5.0, $b$=3.0) compared to the initial values.

In [ ]:
print(w)
print(b)

To further optimize `w` and `b`, we repeat the above process multiple times.

In [ ]:
w = torch.tensor(1.0, requires_grad=True)   # you can set any number here
b = torch.tensor(-5.0, requires_grad=True)  # you can set any number here

lr = 0.01    # learning rate
epochs = 100  # how many times we repeat training

w = torch.tensor(3.0, requires_grad=True)    # initialize w
b = torch.tensor(-1.0, requires_grad=True)   # initialize b

# empty lists for saving loss, w, b
loss_list = []
w_list = []
b_list = []

for epoch in range(epochs):

    p = model(x)              # prediction
    loss = loss_func(p, y)    # measure loss
    loss.backward()           # determine gradients

    with torch.no_grad():     # disable autograd
        w -= w.grad*lr        # update w
        b -= b.grad*lr        # update b

        w.grad.zero_() # reset gradient
        b.grad.zero_() # reset gradient

    # save loss, w, b
    loss_list.append(loss.item())
    w_list.append(w.item())
    b_list.append(b.item())

    print(f"Epoch {epoch+1}, loss={loss.item():.3f}, w={w.item():.3f}, b={b.item():.3f}")

As you can see now, `w` is now close to 5.0, and `b` is close to 3.0.

Let's visualize how the loss value has decreased.

In [ ]:
plt.plot(np.arange(epochs)+1, loss_list)
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.grid()

Let's visualize how the `w` value has changed.

In [ ]:
plt.plot(np.arange(epochs)+1, w_list)
plt.xlabel("Epochs")
plt.ylabel("w")
plt.grid()

Let's visualize how the `b` value has changed.

In [ ]:
plt.plot(np.arange(epochs)+1, b_list)
plt.xlabel("Epochs")
plt.ylabel("b")
plt.grid()

With the optimized values of `w` and `b`, the model fits the dataset very well.

In [ ]:
plt.plot(x, y, marker="o", lw=0, label="dataset")
plt.plot(x, p.detach().numpy(), label="prediction")
plt.grid()
plt.legend()
plt.xlabel("x")
plt.ylabel("y")

Now you can try to change initial values of `epochs`, `lr`, `w`, `b` etc., and let's observe the results.